# Lesson 2: Sequential Chats and Customer Onboarding

## Setup

- a sequence of chats between multiple agents who collaborate to provide a fun customer onboarding experince for a product
- also experince how humans could be seamlessly involved in the loop of an AI system.

In [2]:
# !pip install autogen

In [3]:
# reset all those keys
import os
openai_api_key = os.getenv("OPENAI_API_KEY")

In [4]:
llm_config={"model": "gpt-3.5-turbo"}

In [7]:
# !pip install dask

In [8]:
from autogen import ConversableAgent

## Creating the needed agents

In [9]:
onboarding_personal_information_agent = ConversableAgent(
    name="Onboarding Personal Information Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's name and location.
    Do not ask for other information. Return 'TERMINATE'
    when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)
# sequential task
# customer onboarding task
## collect customer info
## survey customer's topic interest
## customer engagement
## sequential Chat
## Agent A chat with Agent B --> summarizer --> new message
## Agent A chat with Agent B --> summarizer --> new message
## Agent A chat with Agent B --> summarizer --> new message
## ...

In [10]:
onboarding_topic_preference_agent = ConversableAgent(
    name="Onboarding Topic preference Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's preferences on news topics.
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [11]:
customer_engagement_agent = ConversableAgent(
    name="Customer Engagement Agent",
    system_message='''You are a helpful customer service agent
    here to provide fun for the customer based on the user's
    personal information and topic preferences.
    This could include fun facts, jokes, or interesting stories.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

In [12]:
customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

## Creating tasks

Now, you can craft a series of tasks to facilitate the onboarding process.

In [13]:
chats = [
    {
        "sender": onboarding_personal_information_agent,
        "recipient": customer_proxy_agent,
        "message":
            "Hello, I'm here to help you get started with our product."
            "Could you tell me your name and location?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Return the customer information "
                             "into as JSON object only: "
                             "{'name': '', 'location': ''}",
        },
        "max_turns": 2,
        "clear_history" : True
    },
    {
        "sender": onboarding_topic_preference_agent,
        "recipient": customer_proxy_agent,
        "message":
                "Great! Could you tell me what topics you are "
                "interested in reading about?",
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history" : False
    },
    {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "Let's find something fun to read.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
    },
]

In [ ]:
# summary

- a conversational flow where multiple agents, with the goal of gathering information from the user (such as name, location, topics of interest) and transitioning them to different stages of engagement.
- use an LLM for summarization, AI-driven processing of the user's input, transforming it into structured data (like a JSON object).
- and each interaction specifies whether chat history is cleared, how many turns the interaction lasts, and how information is summarized, allowing for a modular and flexible dialogue system.

## Start the onboarding process

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different inputs, such as name, location, and preferences.

In [15]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)
# seperate, enter name and location

## Print out the summary

In [41]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

## Print out the cost

In [36]:
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")